## Importing

In [3]:
import tensorflow as tf

## Tensors
## 3 # a rank 0 tensor; a scalar with shape []
## [1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
## [[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
## [[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

## The Computational Graph

### You might think of TensorFlow Core programs as consisting of two discrete sections:
###    1. Building the computational graph.
###    2. Running the computational graph.

##### A computational graph is a series of TensorFlow operations arranged into a graph of nodes.

In [4]:
node1 = tf.constant(3.0, dtype=tf.float32) # 
node2 = tf.constant(4.0, dtype=tf.float32)
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [5]:
# To see the values of nodes declare a session and run it. i.e. run the computational graph
sess = tf.Session()
print(sess.run([node1, node2]))
# Now we can see the values

[3.0, 4.0]


### Building more complicated computations by combining Tensor nodes with operations (Operations are also nodes).

In [6]:
node3 = tf.add(node1, node2)
print(node3)

Tensor("Add:0", shape=(), dtype=float32)


In [7]:
print(sess.run(node3))

7.0


#### TensorFlow provides a utility called TensorBoard that can display a picture of the computational graph. 

#####  A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.



In [8]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # shortcut for tf.add()

In [9]:
# Running the above computational graph
print(sess.run(adder_node, {a: 3, b: 7}))

10.0


In [10]:
print(sess.run(adder_node, {a: 3, b: 12}))

15.0


In [11]:
print(sess.run(adder_node, {a: [2, 7], b: [4, 6]}))

[  6.  13.]


#### We can make the computational graph more complex by adding another operation.

In [12]:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a: 5, b: 7}))

36.0


### To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. Variables allow us to add trainable parameters to a graph. They are constructed with a type and initial value:



In [35]:
W = tf.Variable([.3], dtype = tf.float32)
b = tf.Variable([-.3], dtype = tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b # Equation of the line y = mx + C where m = y/x (slope)

#### Constants are initialized when you call tf.constant, and their value can never change. By contrast, variables are not initialized when you call tf.Variable.

#### To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:



In [36]:
init = tf.global_variables_initializer()
sess.run(init)

##### It is important to realize init is a handle to the TensorFlow sub-graph that initializes all the global variables. Until we call sess.run, the variables are uninitialized.

##### Since x is a placeholder, we can evaluate linear_model for several values of x simultaneously as follows:

In [37]:
print(sess.run(linear_model, {x: [1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


#### We've created a model, but we don't know how good it is yet. To evaluate the model on training data, we need a y placeholder to provide the desired values, and we need to write a loss function.



#### A loss function measures how far apart the current model is from the provided data. We'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data.

#### SOLUTION
#### linear_model - y creates a vector where each element is the corresponding example's error delta. We call tf.square to square that error. Then, we sum all the squared errors to create a single scalar that abstracts the error of all examples using tf.reduce_sum

In [38]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas) # Calculating the Mean Squared Error or RSS(Residual Sum of squares)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))
# For a theory on Linear Regression/Maths refer https://medium.com/towards-data-science/simple-and-multiple-linear-regression-in-python-c928425168f9

23.66


#### We could improve this manually by reassigning the values of W and b to the perfect values of -1 and 1. A variable is initialized to the value provided to tf.Variable but can be changed using operations like tf.assign.

In [39]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


#### We guessed the "perfect" values of W and b, but the whole point of machine learning is to find the correct model parameters automatically. We will show how to accomplish this in the next section.

### tf.trainAPI

#### TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. The simplest optimizer is `gradient descent`.  GRADIENT DESCENT (MATHS) https://en.wikipedia.org/wiki/Gradient_descent

#### It modifies each variable according to the magnitude of the derivative of loss with respect to that variable. In general, computing symbolic derivatives manually is tedious and error-prone. 

#### Consequently, TensorFlow can automatically produce derivatives given only a description of the model using the function tf.gradients

In [40]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [41]:
sess.run(init) # reset values to incorrect defaults
for i in range(1000):
    sess.run(train, {x: [1,2,3,4], y: [0, -1, -2, -3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


## Complete Program of the above explanation

In [52]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]
# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


## tf.estimator

### tf.estimator is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:

### 1.running training loops
### 2.running evaluation loops
### 3.managing data sets

#### tf.estimator defines many common models.

### Basic usage (TODO: Need to understand the code)

In [53]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np
import tensorflow as tf

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]


# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_master': '', '_save_checkpoints_steps': None, '_is_chief': True, '_keep_checkpoint_max': 5, '_task_id': 0, '_service': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_model_dir': '/tmp/tmpmydoje1d', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f39f521fa20>, '_session_config': None}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpmydoje1d/model.ckpt.
INFO:tensorflow:loss = 10.0, step = 1
INFO:tensorflow:global_step/sec: 649.401
INFO:tensorflow:loss = 0.1296, step = 101 (0.155 sec)
INFO:tensorflow:global_step/sec: 843.456
INFO:tensorflow:loss = 0.0399047, step = 201 (0.120 sec)
INFO:tensorflow:global_step/sec: 740.968
INFO:tensorflow:loss = 0.01085